In [ ]:

import pandas as pd
import numpy as np
from random import random
from sklearn.linear_model import Lasso, lasso_path
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.cm as cm



In [ ]:

memo_time = []
import timeit

def unit(x):
    if x >= 1: return "%1.2f s" % x
    elif x >= 1e-3: return "%1.2f ms" % (x* 1000)
    elif x >= 1e-6: return "%1.2f µs" % (x* 1000**2)
    elif x >= 1e-9: return "%1.2f ns" % (x* 1000**3)
    else:
        return "%1.2g s" % x

def timeexe(legend, code, number=10, repeat=10):
    rep = timeit.repeat(code, number=number, repeat=repeat, globals=globals())
    ave = sum(rep) / (number * repeat)
    std = (sum((x/number - ave)**2 for x in rep) / repeat)**0.5
    fir = rep[0]/number
    fir3 = sum(rep[:3]) / (3 * number)
    las3 = sum(rep[-3:]) / (3 * number)
    rep.sort()
    mini = rep[len(rep)//20] / number
    maxi = rep[-len(rep)//20] / number
    print("Moyenne: %s Ecart-type %s (with %d runs) in [%s, %s]" % (
                unit(ave), unit(std), number, unit(mini), unit(maxi)))
    return dict(legend=legend, average=ave, deviation=std, first=fir, first3=fir3,
                last3=las3, repeat=repeat, min5=mini, max5=maxi, code=code, run=number)




<h2 id="0/-Import-data"> Import data<a class="anchor-link" href="#0/-Import-data">¶</a></h2><p>Motivation Paper: <a href="https://arxiv.org/pdf/1411.6520v1.pdf">https://arxiv.org/pdf/1411.6520v1.pdf</a></p>


In [ ]:

filename = "path"
key = "clean_data"
Data = pd.read_hdf(filename,key)
Data = Data.select_dtypes(include='number')
Data.head(3)



In [ ]:

print(Data.shape)
print()
print(Data.dtypes)
print()
print(Data.columns)
print()
print(Data.isnull().sum())



In [ ]:

X = Data.drop('likes', axis=1)
y = Data['likes']




<h2 id="1/-Baseline-model">Baseline model<a class="anchor-link" href="#1/-Baseline-model">¶</a></h2><ul>
<li><a href="https://www.geeksforgeeks.org/implementation-of-lasso-regression-from-scratch-using-python/">Lasso from scratch</a></li>
<li><a href="https://stats.stackexchange.com/questions/123672/coordinate-descent-soft-thresholding-update-operator-for-lasso">Lasso theory </a></li>
<li><a href="https://xavierbourretsicotte.github.io/lasso_implementation.html">Gradient descent for Lasso</a></li>
<li><a href="https://github.com/satopirka/Lasso/blob/master/coordinate_descent_lasso.py">Gradient descent code</a></li>
</ul>


In [ ]:

def soft_threshold(rho, alpha):
    if rho < -alpha:
        return (rho + alpha)
    elif rho >  alpha:
        return (rho - alpha)
    else: 
        return 0



In [ ]:

def Baseline_Lasso(X, y, max_iter, alpha, intercept=False):
    
    n = X.shape[0]
    
    if intercept == True:
        X_train.insert(0, "intercept", 1)
    
    m = X.shape[1]
    beta = np.zeros(m)
    tmp_beta = np.zeros(m)
    
    for i in range(max_iter):
        
        start = 1 if intercept == True else 0
        
        for j in range(start, m):
            loss_j = y - np.dot(X, tmp_beta)
            rho = np.dot(X.iloc[:, j], loss_j)
            beta[j] = soft_threshold(rho, alpha) / (X.iloc[:, j]**2).sum()

    if intercept == True:
        beta[0] = np.sum(y - np.dot(X.iloc[:, 1:], beta[1:])) / (X.shape[0])

    return beta



In [ ]:

Baseline_Lasso(X, y, alpha=0.1, max_iter=500, intercept=False)



In [ ]:

%timeit Baseline_Lasso(X, y, alpha=0.1, max_iter=500)



In [ ]:

memo_time.append(timeexe("Numpy", "Baseline_Lasso(X, y, alpha=0.1, max_iter=500)"))




<h2 id="2/-Scikit-Learn-optimization">Scikit-Learn optimization<a class="anchor-link" href="#2/-Scikit-Learn-optimization">¶</a></h2>


In [ ]:

regLasso = Lasso(fit_intercept=False, normalize=True)
regLasso.fit(X, y)
print(regLasso.coef_)



In [ ]:

%timeit regLasso.fit(X, y)



In [ ]:

memo_time.append(timeexe("SkLearn", "regLasso.fit(X, y)"))




<h2 id="3/-Cython-optimization"> Cython optimization<a class="anchor-link" href="#3/-Cython-optimization">¶</a>


In [ ]:

%load_ext cython




<h3 id="3.1)-Version-homemade">homemade<a class="anchor-link" href="#3.1)-Version-homemade">¶</a></h3>


In [ ]:

%%cython
cimport cython
cimport numpy as cnp
import numpy as np


@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)
cdef double soft_threshold_c1(double rho, double alpha) nogil:
    if rho < -alpha:
        return (rho + alpha)
    elif rho > alpha:
        return (rho - alpha)
    else:
        return 0.0


@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)
cdef double[:] loss_c(double[:] y, double[:] y_pred, double[:] loss) nogil:
        
    for i in range(0, len(y)):
        loss[i] += y[i] - y_pred[i]
        
    return loss


@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)
cdef double[:] y_pred_c(double[:, :] X, double[:] tmp_beta, double[:] y_pred, int j) nogil:
    
    cdef:
        int n = X.shape[0]
        int m = X.shape[1]
        
    for i in range(0, n):
        y_pred[i] += X[i, j] * tmp_beta[j]
            
    return y_pred


@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)
cdef double rho_c(double[:, :] X, double[:] y_pred, double rho, int j) nogil:
    
    cdef int n = X.shape[0]
        
    for i in range(0, n):
        rho += X[i, j] * y_pred[i]
        
    return rho


@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)
cdef double normalisation_c(double[:, :] X, double X_nrm, int j) nogil:
    
    cdef int n = X.shape[0]
        
    for i in range(0, n):
        X_nrm += X[i, j]**2
        
    return X_nrm


@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)
cdef double[:] fit1(double[:, :] X, double[:] y, int max_iter, float alpha,
                   int n, int m, double[:] beta, double[:] tmp_beta,
                   double[:] y_pred, double[:] loss, double X_nrm, double rho):
    
    cdef int i, j

    for i in range(max_iter):

        for j in range(m):
            
            y_pred_c(X, tmp_beta, y_pred, j)
            loss_c(y, y_pred, loss)
            rho_c(X, y_pred, rho, j)
            beta[j] = soft_threshold_c1(rho, alpha) / normalisation_c(X, X_nrm, j) #(X.iloc[:, j]**2).sum()
            
    return beta


@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)
def Cython_Lasso1(double[:, :] X, double[:] y, float alpha, int max_iter):

    cdef: #initialisation
        int n = X.shape[0]
        int m = X.shape[1]
        double[:] beta = np.zeros(m, dtype=np.float64)
        double[:] tmp_beta = np.zeros(m, dtype=np.float64)
        double[:] y_pred = np.empty(n, dtype=np.float64)
        double[:] loss = np.empty(n, dtype=np.float64)
        double X_nrm = 0.0
        double rho = 0.0
        
        sol = fit1(X, y, max_iter, alpha, n, m, beta, tmp_beta, y_pred, loss, X_nrm, rho)

    return np.asarray(sol)



In [ ]:

X_val = X.values.astype(np.float64)
y_val = y.values.astype(np.float64)
Cython_Lasso1(X_val, y_val, alpha=0.1, max_iter=500)



In [ ]:

%timeit Cython_Lasso1(X_val, y_val, alpha=0.1, max_iter=500)



In [ ]:

memo_time.append(timeexe("Cython_homemade", "Cython_Lasso1(X_val, y_val, alpha=0.1, max_iter=500)"))




<h3 id="3.2)-Version-library-Blas">Blas<a class="anchor-link" href="#3.2)-Version-library-Blas">¶</a></h3>


In [ ]:

%%cython
cimport cython
cimport numpy as np
import numpy as np
from libc.math cimport fabs, sqrt
from scipy.linalg.cython_blas cimport ddot, dasum, daxpy, dnrm2, dcopy, dscal


cdef double soft_threshold_c2(double f) nogil:
    if f == 0:
        return 0
    elif f > 0:
        return 1.0
    else:
        return -1.0


@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)
cdef double[:] fit2(double[::1, :] X, double[:] norms_X_col, double[:] y,
                   double[:] beta_tmp, double[:] beta, double alpha,
                   int n, int m, int max_iter) nogil:

    cdef:
        int inc = 1
        double tmp
        double mbetaj

    dcopy(&n, &y[0], &inc, &beta_tmp[0], &inc) #R[0] <- y[0]
    for i in range(max_iter):

        for j in range(m):

            if beta[j] != 0.:
                daxpy(&n, &beta[j], &X[0, j], &inc, &beta_tmp[0], &inc) #R += X[:, j] * beta[j]
                
            tmp = ddot(&n, &beta_tmp[0], &inc, &X[0, j], &inc) #tmp = X[:, j].dot(R)
            beta[j] = soft_threshold_c2(tmp) * max(fabs(tmp) - alpha, 0) / norms_X_col[j] # l1 penalisation

            if beta[j] != 0.:
                mbetaj = - beta[j]
                daxpy(&n, &mbetaj, &X[0, j], &inc, &beta_tmp[0], &inc) #R += - beta[j] * X[:, j]
                
    return beta


@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)
def Cython_Lasso2(double[::1, :] X, double[:] y, float alpha, int max_iter=200):

    cdef:
        int n = X.shape[0]
        int m = X.shape[1]
        int inc = 1

    cdef: #initialisation
        double[:] sol
        double[:] beta_tmp = np.empty(n, dtype=np.float64)
        double[:] beta = np.zeros(m, dtype=np.float64)
        double[:] norms_X_col = np.empty(m, dtype=np.float64)

    for j in range(m):
        norms_X_col[j] = ddot(&n, &X[0, j], &inc, &X[0, j], &inc)
    with nogil:
            
        sol = fit2(X, norms_X_col, y, beta_tmp, beta, alpha, n, m, max_iter)

    return np.asarray(sol)



In [ ]:

import numpy as np
X_val = X.values.astype(np.float64)
y_val = y.values.astype(np.float64)
Cython_Lasso2(X_val, y_val, alpha=0.1, max_iter=500)



In [ ]:

%timeit Cython_Lasso2(X_val, y_val, alpha=0.1, max_iter=500)



In [ ]:

memo_time.append(timeexe("Cython_Blas", "Cython_Lasso2(X_val, y_val, alpha=0.1, max_iter=500)"))




<h2 id="4/-Parallel-features-optimization">Parallel-features optimization<a class="anchor-link" href="#4/-Parallel-features-optimization">¶</a></h2><ul>
<li><a href="https://stackoverflow.com/questions/59250916/can-this-problem-be-implemented-in-parallel-in-cython-with-openmp">stackoverflow</a></li>
<li><a href="https://github.com/IlyaTrofimov/dlr">Original code</a></li>
</ul>



<h3 id="4.1)-Version-homemade-parallélisée"> homemade<a class="anchor-link" href="#4.1)-Version-homemade-parallélisée">¶</a></h3>


In [ ]:

%%cython
cimport cython
cimport numpy as cnp
import numpy as np
from cython.parallel import parallel, prange


@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)
cdef double soft_threshold_c1(double rho, double alpha) nogil:
    if rho < -alpha:
        return (rho + alpha)
    elif rho > alpha:
        return (rho - alpha)
    else:
        return 0.0


@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)
cdef double[:] loss_c(double[:] y, double[:] y_pred, double[:] loss) nogil:
        
    for i in range(0, len(y)):
        loss[i] += y[i] - y_pred[i]
        
    return loss


@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)
cdef double[:] y_pred_c(double[:, :] X, double[:] tmp_beta, double[:] y_pred, int j) nogil:
    
    cdef:
        int n = X.shape[0]
        int m = X.shape[1]
        
    for i in range(0, n):
        y_pred[i] += X[i, j] * tmp_beta[j]
            
    return y_pred


@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)
cdef double rho_c(double[:, :] X, double[:] y_pred, double rho, int j) nogil:
    
    cdef int n = X.shape[0]
        
    for i in range(0, n):
        rho += X[i, j] * y_pred[i]
        
    return rho


@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)
cdef double normalisation_c(double[:, :] X, double X_nrm, int j) nogil:
    
    cdef int n = X.shape[0]
        
    for i in range(0, n):
        X_nrm += X[i, j]**2
        
    return X_nrm


@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)
cdef double[:] Parallel_fit1(double[:, :] X, double[:] y, int max_iter, float alpha,
                             int n, int m, double[:] beta, double[:] tmp_beta,
                             double[:] y_pred, double[:] loss, double X_nrm, double rho):

    cdef int i, j

    for i in range(max_iter):

        for j in prange(m, nogil=True):
            
            y_pred_c(X, tmp_beta, y_pred, j)
            loss_c(y, y_pred, loss)
            rho_c(X, y_pred, rho, j)
            beta[j] = soft_threshold_c1(rho, alpha) / normalisation_c(X, X_nrm, j) #(X.iloc[:, j]**2).sum()
            
    return beta


@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)
def Parallel_Cython_Lasso1(double[:, :] X, double[:] y, float alpha, int max_iter):

    cdef: #initialisation
        int n = X.shape[0]
        int m = X.shape[1]
        double[:] beta = np.zeros(m, dtype=np.float64)
        double[:] tmp_beta = np.zeros(m, dtype=np.float64)
        double[:] y_pred = np.empty(n, dtype=np.float64)
        double[:] loss = np.empty(n, dtype=np.float64)
        double X_nrm = 0.0
        double rho = 0.0
        
        sol = Parallel_fit1(X, y, max_iter, alpha, n, m, beta, tmp_beta, y_pred, loss, X_nrm, rho)

    return np.asarray(sol)



In [ ]:

X_val = X.values.astype(np.float64)
y_val = y.values.astype(np.float64)
Parallel_Cython_Lasso1(X_val, y_val, alpha=0.1, max_iter=500)



In [ ]:

%timeit Parallel_Cython_Lasso1(X_val, y_val, alpha=0.1, max_iter=500)



In [ ]:

memo_time.append(timeexe("Parallel_Cython_Homemade", "Parallel_Cython_Lasso1(X_val, y_val, alpha=0.1, max_iter=500)"))




<h3 id="4.2)-Version-Blas-parallélisée">Blas<a class="anchor-link" href="#4.2)-Version-Blas-parallélisée">¶</a></h3>


In [ ]:

%%cython
cimport cython
cimport numpy as np
import numpy as np
from libc.math cimport fabs, sqrt
from scipy.linalg.cython_blas cimport ddot, dasum, daxpy, dnrm2, dcopy, dscal
from cython.parallel import parallel, prange


cdef double soft_threshold_c2(double f) nogil:
    if f == 0:
        return 0
    elif f > 0:
        return 1.0
    else:
        return -1.0


@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)
cdef double[:] Parallel_fit2(double[::1, :] X, double[:] norms_X_col, double[:] y,
                            double[:] R, double[:] beta, double alpha,
                            int n, int m, int max_iter):

    cdef:
        int inc = 1
        int j
        double tmp
        double mbetaj

    dcopy(&n, &y[0], &inc, &R[0], &inc) #R[0] <- y[0]
    for i in range(max_iter):

        for j in prange(m, nogil=True):

            if beta[j] != 0.:
                daxpy(&n, &beta[j], &X[0, j], &inc, &R[0], &inc) #R += X[:, j] * beta[j]
                
            tmp = ddot(&n, &R[0], &inc, &X[0, j], &inc) #tmp = X[:, j].dot(R)
            beta[j] = soft_threshold_c2(tmp) * max(fabs(tmp) - alpha, 0) / norms_X_col[j] # l1 penalisation

            if beta[j] != 0.:
                mbetaj = - beta[j]
                daxpy(&n, &mbetaj, &X[0, j], &inc, &R[0], &inc) #R += - beta[j] * X[:, j]
                
    return beta


@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)
def Parallel_Cython_Lasso2(double[::1, :] X, double[:] y, float alpha, int max_iter):

    cdef:
        int n = X.shape[0]
        int m = X.shape[1]
        int inc = 1

    cdef: #initialisation
        double[:] sol
        double[:] R = np.empty(n, dtype=np.float64)
        double[:] beta = np.zeros(m, dtype=np.float64)
        double[:] norms_X_col = np.empty(m, dtype=np.float64)
    
    for j in range(m):
        norms_X_col[j] = ddot(&n, &X[0, j], &inc, &X[0, j], &inc)
    
    sol = Parallel_fit2(X, norms_X_col, y, R, beta, alpha, n, m, max_iter)

    return np.asarray(sol)



In [ ]:

import numpy as np
X_val = X.values.astype(np.float64)
y_val = y.values.astype(np.float64)
Parallel_Cython_Lasso2(X_val, y_val, alpha=0.1, max_iter=500)



In [ ]:

%timeit Parallel_Cython_Lasso2(X_val, y_val, alpha=0.1, max_iter=500)



In [ ]:

memo_time.append(timeexe("Parallel_Cython_Blas", "Parallel_Cython_Lasso2(X_val, y_val, alpha=0.1, max_iter=500)"))




<h2 id="5/-Algorithms-comparison">Algorithms comparison<a class="anchor-link" href="#5/-Algorithms-comparison">¶</a></h2>


In [ ]:

df = pd.DataFrame(data=memo_time)
df = df.set_index("legend").sort_values("average")
cols = ["average", "deviation", "min5", "max5", "run", "code"]
df[cols]



In [ ]:

fig, ax = plt.subplots(1, 1, figsize=(14,6))
df[["average", "deviation"]].plot(kind="barh", logx=True, ax=ax, xerr="deviation",
                                  legend=False, fontsize=12, width=0.8)
ax.set_ylabel("")
ax.grid(b=True, which="major")
ax.grid(b=True, which="minor")

